In [52]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [53]:
import os
import plotly
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as py 
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from builder import load_surv_samples

import theano
import theano.tensor as T

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import itemfreq

from sklearn.model_selection import train_test_split

/Users/bychkov/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1401: UserWarning:

 This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.


Using TensorFlow backend.


## Data:

In [ ]:
data_x, data_y = load_surv_samples('simulated_data/linear_5K.pkl', sort=False)

train_x, test_x, train_y, test_y = train_test_split(
        data_x, data_y, test_size=0.2, random_state=75 )

train_x, val_x, train_y, val_y = train_test_split(
        train_x, train_y, test_size=0.25, random_state=75 )

print("Train split: %d; Val split: %d; Test split: %d." % (train_x.shape[0],val_x.shape[0],test_x.shape[0]))

In [ ]:
print( test_x.shape )
print( test_y.shape )

### Partial likelihood [ Theano ]

In [54]:
def partial_likelihood_th(y_true, y_pred):
    sort_idx = np.argsort( y_true[:,1] )[::-1]

    risk    = y_pred[sort_idx]
    events  = y_true[:,2][sort_idx]

    hazard_ratio = T.exp(risk)
    log_cum_risk = T.log(T.extra_ops.cumsum(hazard_ratio))
    uncencored_likelihood = risk.T - log_cum_risk
    censored_likelihood = uncencored_likelihood * events
    neg_likelihood = -T.sum( censored_likelihood )

    return neg_likelihood

### Partial likelihood [ NumPy ]

In [55]:
def partial_likelihood_np(y_true, y_pred):
    # Sort by time: longest -> shortest  i.g. [4,3,2,1]
    sort_idx = np.argsort( y_true[:,1] )[::-1]

    risk    = y_pred[sort_idx]
    events  = y_true[:,2][sort_idx]

    hazard_ratio = np.exp(risk)
    log_cum_risk = np.log(np.cumsum(hazard_ratio))
    uncencored_likelihood = risk - log_cum_risk
    censored_likelihood = uncencored_likelihood * events
    neg_likelihood = -np.sum( censored_likelihood )

    return neg_likelihood

### Efron [ NumPy ]

In [56]:
def efron_estimator_np_vec(y_true, y_pred):
    sort_idx = np.argsort( y_true[:,1] )[::-1]
    
    risk          = y_pred[sort_idx]
    risk_exp      = np.exp(risk)
    events        = y_true[:,2][sort_idx]
    ftimes        = y_true[:,1][sort_idx]
    ftimes_cens   = ftimes * events
    
    unique        = np.unique(ftimes_cens, return_index=True, return_counts=True)
    unique_ftimes = np.trim_zeros( unique[0][::-1] )
    m = np.count_nonzero(unique_ftimes)
    
    E_ti     = np.zeros(m, dtype='int32')
    risk_phi = np.zeros(m, dtype='float32')
    cum_risk = np.zeros(m, dtype='float32')
    tie_phi  = np.zeros(m, dtype='float32')
    
    cum_sum = np.cumsum(risk_exp)
    
    for j in range(m):
        idx = np.logical_and(ftimes == unique_ftimes[j], events)
        E_ti[j]      = idx.sum()
        
        risk_phi[j]  = risk[idx].sum()
        tie_phi[j]   = risk_exp[idx].sum()
        
        cum_risk[j]  = cum_sum[ ftimes == unique_ftimes[j] ][-1]
        
    likelihood = 0.
    for j in range(m):
        J = np.linspace(start=0, stop=E_ti[j]-1, num=E_ti[j]) / np.float(E_ti[j])
        D_m = cum_risk[j] - J*tie_phi[j]
        likelihood += risk_phi[j] - np.log(D_m).sum()
        
    return np.negative(likelihood)

In [57]:
def efron_estimator_np(y_true, y_pred):
    sort_idx = np.argsort( y_true[:,1] )[::-1]
    
    # Sort & prepare:
    risk          = y_pred[sort_idx]
    risk_exp      = np.exp(risk)
    events        = y_true[:,2][sort_idx]
    ftimes        = y_true[:,1][sort_idx]

    # Initis:
    tie_count    = 0
    likelihood   = 0.
    cum_risk, risk_phi, tie_phi = 0., 0., 0. 
    
    # Iterate over samples in inverse-time-order:
    for i, (ti, ei) in list(enumerate(zip(ftimes, events))):
        cum_risk += risk_exp[i]
        
        if ei:
            risk_phi   += risk[i]
            tie_phi    += risk_exp[i]
            tie_count  += 1
            
        do_sum = (i == (ftimes.size - 1) and tie_count > 0) or \
                 (ftimes[i + 1] < ti and tie_count > 0)
        if  do_sum:
            # Diagnostic print #1
            #print_function('%f - '%(risk_phi))
            
            for l in range(tie_count):
                c    = l / float(tie_count)
                dm   = np.log(cum_risk - c * tie_phi)
                likelihood -= dm
                
                # Diagnostic print #2
                #print('  log(%f - %f * %f) ' % (cum_risk, c, tie_phi))

            likelihood += risk_phi
            
            # Diagnostic print #3
            #print( '%i) %f %f %f, %i' % (i, risk_phi, cum_risk, tie_phi, tie_count) )

            # Reset:
            tie_phi   = 0.
            risk_phi  = 0.
            tie_count = 0
        
    return np.negative(likelihood)

### Efron [ TensorFlow]

In [58]:
import tensorflow as tf
sess = tf.Session()

import keras
from keras import backend as K
K.set_session(sess)

In [59]:
def efron_estimator_tf(y_true, y_pred):
    sort_idx = tf.nn.top_k(y_true[:,1], k=tf.shape(y_pred)[0], sorted=True).indices
    
    risk          = tf.gather(y_pred, sort_idx)
    risk_exp      = tf.exp(risk)
    events        = tf.gather(y_true[:,2], sort_idx)
    ftimes        = tf.gather(y_true[:,1], sort_idx)
    ftimes_cens   = ftimes * events
    
    # Get unique failure times & Exclude zeros 
    # NOTE: this assumes that falure times start from > 0 (greater than zero)
    unique = tf.unique(ftimes_cens).y
    unique_ftimes = tf.boolean_mask(unique, tf.greater(unique, 0) )
    m = tf.shape(unique_ftimes)[0]
    
    # Define key variables:
    log_lik  = tf.Variable(0., dtype=tf.float32, validate_shape=False, trainable=False)
    E_ti     = tf.Variable([], dtype=tf.int32,   validate_shape=False, trainable=False)
    risk_phi = tf.Variable([], dtype=tf.float32, validate_shape=False, trainable=False)
    tie_phi  = tf.Variable([], dtype=tf.float32, validate_shape=False, trainable=False)
    cum_risk = tf.Variable([], dtype=tf.float32, validate_shape=False, trainable=False)
    cum_sum  = tf.cumsum(risk_exp)
    
    # -----------------------------------------------------------------
    # Prepare for looping:
    # -----------------------------------------------------------------
    i = tf.constant(0, tf.int32)
    def loop_cond(i, *args):
        return i < m

    # Step for loop # 1:
    def loop_1_step(i, E, Rp, Tp, Cr, Cs):
        n = tf.shape(Cs)[0]
        idx_b = tf.logical_and(
            tf.equal(ftimes, unique_ftimes[i]), 
            tf.equal(events, tf.ones_like(events)) )
        
        idx_i = tf.cast(
            tf.boolean_mask( 
                tf.lin_space(0., tf.cast(n-1,tf.float32), n), 
                tf.greater(tf.cast(idx_b, tf.int32),0)
            ), tf.int32 )
        
        E  = tf.concat([E, [tf.reduce_sum(tf.cast(idx_b, tf.int32))]], 0)
        Rp = tf.concat([Rp, [tf.reduce_sum(tf.gather(risk, idx_i))]], 0)
        Tp = tf.concat([Tp, [tf.reduce_sum(tf.gather(risk_exp, idx_i))]], 0)
        
        idx_i = tf.cast(
            tf.boolean_mask( 
                tf.lin_space(0., tf.cast(n-1,tf.float32), n), 
                tf.greater(tf.cast(tf.equal(ftimes, unique_ftimes[i]), tf.int32),0)
            ), tf.int32 )
        
        Cr = tf.concat([Cr, [tf.reduce_max(tf.gather( Cs, idx_i))]], 0) 
        return i + 1, E, Rp, Tp, Cr, Cs
    
    # Step for loop # 1:
    def loop_2_step(i, E, Rp, Tp, Cr, likelihood):
        l = E_ti[i]
        J = tf.lin_space(0., tf.cast(l-1,tf.float32), l) / tf.cast(l, tf.float32)
        Dm = Cr[i] - J * Tp[i]
        likelihood = likelihood + Rp[i] - tf.reduce_sum(tf.log(Dm))
        return i + 1, E, Rp, Tp, Cr, likelihood
    
    # -----------------------------------------------------------------
    
    # Loop # 1:
    _, E_ti, risk_phi, tie_phi, cum_risk, _ = loop_1 = tf.while_loop(
        loop_cond, loop_1_step,
        [i, E_ti, risk_phi, tie_phi, cum_risk, cum_sum]
    )
    
    # Loop # 2:
    loop_2 = tf.while_loop(
        loop_cond, loop_2_step,
        [i, E_ti, risk_phi, tie_phi, cum_risk, log_lik]
    )
    
    log_lik = loop_2[5]
    return tf.negative(log_lik)

## Lousy Testing

In [60]:
# dim order in Y: 
# 0  1  2  3
# h, t, e, id
hs     = np.array([0]*4)
ts     = np.array([1,3,4,5])
es     = np.array([1,1,1,1])
id     = np.array([0]*4)
preds  = np.array([0.1,0.18,0.3,0.4])

ys_uncens = np.column_stack((hs,ts,es,id))

print( partial_likelihood_th(ys_uncens, preds).eval() )
print( partial_likelihood_np(ys_uncens, preds) )
print( efron_estimator_np_vec(ys_uncens, preds) )
print( efron_estimator_np(ys_uncens, preds) )
K.eval( efron_estimator_tf(K.variable(ys_uncens), K.variable(preds)) )

3.49821419494
3.49821419494
3.49821411527
3.49821419494


3.498214

In [61]:
# dim order in Y: 
# 0  1  2  3
# h, t, e, id
hs     = np.array([.1]*9)
ts     = np.array([1,2,3,4,5,5,7,1,1])
es     = np.array([1,1,0,1,1,1,1,0,1])
id     = np.array([0]*9)
preds  = np.array([-0.4, -0.3, -0.2, -0.1,  0.0,  0.1,  0.2,  0.3,  0.4])

ys_uncens = np.column_stack((hs,ts,es,id))

print( partial_likelihood_th(ys_uncens, preds).eval() )
print( partial_likelihood_np(ys_uncens, preds) )
print( efron_estimator_np_vec(ys_uncens, preds) )
print( efron_estimator_np(ys_uncens, preds) )
K.eval( efron_estimator_tf(K.variable(ys_uncens), K.variable(preds)) )

9.76101946041
9.76101946041
9.85944477328
9.85944496367


9.8594446

## Proper tests:

In [62]:
import pandas as pd
from rpy2 import robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.vectors import DataFrame
from rpy2 import robjects as ro
pandas2ri.activate()

In [136]:
# Define R-code to compute efron log-likelihood:
r_code = '''
    function(test.d){
        fit <- coxph(Surv(time, status) ~ x1 + x2, test.d, method = 'efron', init = c(0.9,1.5), iter.max=0) 
        out = list( fit$linear.predictors, fit$loglik )
    }
'''

rfunc  = robjects.r(r_code)

In [142]:
N = 1024 # np.random.randint(15,25)
tie_ratio   = 0.7 # [0,1]
set_size    = int(np.round(N*tie_ratio))
censor_rate = 0.5

hs     = np.array([.1]*set_size)
ts     = np.linspace(1, N, N)  # + np.random.normal(0., 0.05, N)
ts     = ts[ np.random.choice(N, set_size, replace=True) ]
es     = np.random.binomial(1, (1-censor_rate), set_size)
id     = np.array([0]*set_size)

y_data = np.column_stack((hs,ts,es,id))

# Create a data-frame for R:
df = pd.DataFrame({
        'time'   : ts,
        'status' : es,
        'x1'     : np.random.uniform(-1.0, 1.0, set_size),
        'x2'     : np.random.uniform(-1.0, 1.0, set_size) })

# Compute likelihood with R:
r_out  = rfunc( df )
preds, r_lik  = r_out[0], np.negative(np.round(r_out[1][0],4))


print( '__: ', np.round(partial_likelihood_np(y_data, preds),4) )
print( 'Np: ', np.round(efron_estimator_np_vec(y_data, preds),4) )
print( 'TF: ', K.eval( efron_estimator_tf(K.variable(y_data), K.variable(preds)) ) )
print( 'R : ', r_lik )

__:  2155.118
Np:  2154.591
TF:  2154.59
R :  2154.591


## From R-simulated data:

In [ ]:
N = 100
s_idx = np.random.choice(1000, N)
y_true = test_y[s_idx,...]
y_pred = np.random.uniform(0.0, 0.1, N)

print( partial_likelihood_th(ys_uncens, preds).eval() )
print( partial_likelihood_np(ys_uncens, preds) )
print( efron_estimator_np_vec(ys_uncens, preds) )
print( efron_estimator_np(ys_uncens, preds) )
K.eval( efron_estimator_tf(K.variable(ys_uncens), K.variable(preds)) )